<a href="https://colab.research.google.com/github/Vikram-Ahuja/Stock_Portfolios/blob/main/Python_Bot_freecodecamp_quantitative_momentum_strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import requests
import math
from scipy import stats

In [ ]:
stocks = pd.read_csv("/content/drive/MyDrive/sp_500_stocks.csv")
api_token = "186b12d3aeb74b95ac7886dd0c1e82cc"
symbol = "AAPL"
base_url =f"https://api.twelvedata.com/price?symbol={symbol}&apikey={api_token}"
data = requests.get(base_url).json()

data

{'price': '121.23000'}

In [ ]:
My_col = ['Ticker', 'Price', 'One-Year_Price_Return', 'Number_of_Shares_to_Buy']
My_Stock_Momentum_Data = pd.DataFrame(columns=My_col)

My_Stock_Momentum_Data

,Ticker,Price,One-Year_Price_Return,Number_of_Shares_to_Buy


In [ ]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

for symbol_string in symbol_strings:
    batch_url = f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={api_token}"
    data = requests.get(batch_url).json()
    for symbol in symbol_string.split(","):
        My_Stock_Momentum_Data = My_Stock_Momentum_Data.append(
            pd.Series(
                [
                 symbol,
                 data[symbol]["price"],
                 data[symbol]["stats"]["year1ChangePercent"],
                 "N/A"
                ],
                index=My_col
            ),
            ignore_index=True
        )
        

In [ ]:
My_Stock_Momentum_Data

In [ ]:
My_Stock_Momentum_Data = My_Stock_Momentum_Data.sort_values('One-Year_Price_Return',ascending=False,ignore_index=True)
My_Stock_Momentum_Data = My_Stock_Momentum_Data[:50]

My_Stock_Momentum_Data

In [ ]:
My_Portfoliio_size = int(input("Enter Your Size : "))
position_size = My_Portfoliio_size/len(My_Stock_Momentum_Data)

for ind in range(len(My_Stock_Momentum_Data)):
    no_stocks_to_buy = math.floor(position_size/My_Stock_Momentum_Data.Price[ind])
    My_Stock_Momentum_Data.Number_of_Shares_to_Buy[ind] = no_stocks_to_buy  

In [ ]:
My_Stock_Momentum_Data.to_csv("/content/drive/MyDrive/Python_Momentum_Portfolio.csv")